In [ ]:
import numpy as np
import holoviews as hv
hv.extension('bokeh')
import panel as pn
from tqdm import tqdm
import h5py as h5
from pathlib import Path
import tables
import sys
from importlib import reload

import psana as ps
sys.path.append('/sdf/group/lcls/ds/tools/smalldata_tools/latest')
from smalldata_tools.DetObject import DetObject, DetObjectFunc
from smalldata_tools.ana_funcs.azimuthalBinning import azimuthalBinning

from smalldata_tools.SmallDataUtils import setParameter, getUserData, getUserEnvData

from smalldata_tools.utilities_plotting import hv_image_ctl
from smalldata_tools.utilities_plotting import hv_image
from smalldata_tools.utilities import image_from_dxy

run = 320
exp = 'xpptut15'

dsname = 'exp={}:run={}'.format(exp,run,exp[:3],exp)
ds = ps.MPIDataSource(dsname)

# Set detObject and ana function

This is where the detector and the analysis function are defined. Here we specify the 'LCLS' azimuthal averaging

In [ ]:
det = DetObject('cspad', ds.env(), int(run))

func_dict = {}
func_dict['eBeam'] = 9.8
func_dict['qbin'] = 0.05
func_dict['thresADU'] = 7
#func_dict['thresADUhigh'] = 17.5
func_dict['pPlane'] = 0
func_dict['center'] = [87392.2, -10126.4]
func_dict['dis_to_sam'] = 90.
#func_dict['phiBins'] = 5
    
func = azimuthalBinning(**func_dict)

# add function to detector pipeline
det.addFunc(func)

userDataCfg = det.params_as_dict()

#run this on an event
max_evt = 1
ds.break_after(max_evt) # stop iteration after max_evt events (break statements do not work reliably with MPIDataSource).

userDict = {}
for nevt,evt in tqdm(enumerate(ds.events())):
    det.getData(evt)
        
    det.processFuncs()
    userDict[det._name]=getUserData(det)

    print(userDict)

## Plot the detector data 
The <DetObject>.getData(evt) extracts the detector data from the event using the specified calibration settings and places it into the det.evt.dat object. This data then gets passed to the DetObjectFuncs, but you can also experiment writing your own code in the event loop after the getData call.If you feel you have a new function that could be of use for others (or just to you again when you return), please reach out to add it to smalldata_tools arsenal of supported functions! 

In [ ]:
det_evt_thres = det.evt.dat.copy()
thresADU = 7
det_evt_thres[det.evt.dat<thresADU]=0 
hv_image(image_from_dxy(det_evt_thres,userDataCfg['ix'],userDataCfg['iy']), imgLow=-3.5, imgHigh=170., cmap='viridis')

In [ ]:
hv.Points((0.5*(userDataCfg['azav__azav_qbins'][:-1]+userDataCfg['azav__azav_qbins'][1:]),\
           userDict['cspad']['azav_azav'].sum(axis=0))).options(width=800)

# Getting the UserData to check the setup
This is data that usually will be saved in the hdf5 file under UserDataCfg/<detname>/

In [ ]:
# look at the configuration data - first detector as in example 2
x = userDataCfg['x']
y = userDataCfg['y']

maskImg = image_from_dxy(userDataCfg['cmask'].flatten(), userDataCfg['ix'].flatten(), userDataCfg['iy'].flatten())
maskPlt = hv_image(maskImg, bounds=(x.min(), y.min(), x.max(), y.max()), cmap='viridis')

# look at the configuration data - now data for the data reduction algorithm
use_geomCorr = userDataCfg['azav__azav_geomCorr']
geomCorr = userDataCfg['azav__azav_geom']

use_polCorr = userDataCfg['azav__azav_polCorr']
polCorr = userDataCfg['azav__azav_pol']

geomImg = image_from_dxy(geomCorr, userDataCfg['ix'].flatten(), userDataCfg['iy'].flatten())
geomPlt = hv_image(geomImg, bounds=(x.min(), y.min(), x.max(), y.max()), cmap='rainbow')

polImg = image_from_dxy(polCorr, userDataCfg['ix'].flatten(), userDataCfg['iy'].flatten())
polPlt = hv_image(polImg, bounds=(x.min(), y.min(), x.max(), y.max()), cmap='rainbow')

idxImg = image_from_dxy(userDataCfg['azav__azav_Cake_idxs'], 
                        userDataCfg['ix'].flatten()[userDataCfg['cmask'].flatten()>0], 
                        userDataCfg['iy'].flatten()[userDataCfg['cmask'].flatten()>0])
idxPlt = hv_image(idxImg, bounds=(x.min(), y.min(), x.max(), y.max()), cmap='glasbey')

#assemble this into a tabbed plot
tabs1 = pn.Tabs(idxPlt.options(title='Bin Index'), name='Bin Index')
tabs = pn.Tabs(tabs1)
tabs.append(pn.Tabs(maskPlt.options(title='Mask'), name='Mask'))
tabs.append(pn.Tabs(geomPlt.options(title='Geometry Correction'),name='Geometry Correction'))
tabs.append(pn.Tabs(polPlt.options(title='Polarization Correction'),name='Polarization Correction'))

tabs